In [243]:
import pandas as pd
import numpy as np
import json
from nltk.tokenize import word_tokenize, wordpunct_tokenize
import re
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
import datetime
import matplotlib.pyplot as plt
import os

stops = stopwords.words('russian')
morph = MorphAnalyzer()

In [99]:
def date(date_value):
    value = datetime.datetime.fromtimestamp(date_value)
    year = value.strftime('%Y')
    month = value.strftime('%m')
    day = value.strftime('%d')
    hour = value.strftime('%H')
    return year, month, day, hour

In [181]:
def normalize(text):
    lemmas = []
    text = text.lower()
    text = re.sub('!|\#|\$|%|\&|\(|\)|\*|\||\+|,|-|\.|\'|\/|\:|;|<|=|>|❤|\“|♥|\?|\@|\[|¶|\]|\^|_|\`||\{|\}|~|\«|\»|\«|\»|\"|\…', '', text)
    text = re.sub('(http://|https://|www.)[a-zA-z0-9]*? ', '', text)
    text = re.sub('[A-Za-z0-9]+', '', text)
    text = re.sub('\s+|\n|\t', ' ', text)
    tokens = wordpunct_tokenize(text)
    for el in tokens:
        lemmas.append(morph.parse(el)[0].normal_form)
    
    clean_lemmas = [lemma for lemma in lemmas]
    return clean_lemmas

In [244]:
def write_in_df(el, year, month, day, hour, df):
    pos = morph.parse(el)[0].tag.POS 
    app = {'Лексема': str(el),
           'Часть речи': str(pos),
           'Год': str(year),
           'Месяц': str(month),
           'День': str(day), 
           'Час': str(hour)}
    df = df.append(app, ignore_index=True)
    
    return df

In [245]:
def preprocess(f, df):
    with open('/Users/alinashaymardanova/Desktop/Курсовая/vk_corpus/' + str(f), encoding='utf-8') as file:
        data = json.load(file)
        for el in data:
            year, month, day, hour = date(el['date'])
            for el in normalize(el['text']):
                df = write_in_df(el, year, month, day, hour, df)
    return df

In [254]:
def wite_to_csv(f, df):
    f = re.search('[0-9]+', str(f))[0]
    with open('/Users/alinashaymardanova/Desktop/Курсовая/' + str(f) + '.csv', 'w', encoding='utf-8') as file:
        df.to_csv(file, sep='\t')

Подготавливаем csv файлы для каждого года

In [ ]:
for (path, dirs, files) in os.walk('/Users/alinashaymardanova/Desktop/Курсовая/vk_corpus'):
        for f in files:
            print(f)
            if f != '.DS_Store':
                col = {'Лексема','Часть речи','Год','Месяц','День', 'Час'}
                df = pd.DataFrame(columns=col)
                df = preprocess(f, df)
                wite_to_csv(f, df)

.DS_Store
2008.json
2009.json
2010.json
2011.json
2012.json
2013.json
2014.json
2015.json
